# Import the Datasets

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances, manhattan_distances
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Data Gathered By Me Via the MyAnimeList API and Jikan REST API
anime_df = pd.read_csv('./data/anime.csv')
user_ratings_df = pd.read_csv('./data/user_ratings.csv', parse_dates=['updated_at'])

In [3]:
anime_df.head()

,id,title,start_date,end_date,synopsis,score,rank,popularity,num_list_users,num_scoring_users,media_type,status,genres,num_episodes,start_season,broadcast_day,broadcast_time,source,rating,studios
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1.0,3,3240265,2056814,tv,finished_airing,"Action,Adventure,Drama,Fantasy,Military,Shounen",64,spring,sunday,17:00,manga,r,Bones
1,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.07,2.0,13,2489957,1361494,tv,finished_airing,"Drama,Psychological,Sci-Fi,Suspense,Time Travel",24,spring,wednesday,02:05,visual_novel,pg_13,White Fox
2,28977,Gintama°,2015-04-08,2016-03-30,"Gintoki, Shinpachi, and Kagura return as the f...",9.06,3.0,336,610158,243298,tv,finished_airing,"Action,Comedy,Gag Humor,Historical,Parody,Samu...",51,spring,wednesday,18:00,manga,pg_13,Bandai Namco Pictures
3,41467,Bleach: Sennen Kessen-hen,2022-10-11,2022-12-27,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.05,4.0,432,489199,245969,tv,finished_airing,"Action,Adventure,Fantasy,Shounen",13,fall,tuesday,00:00,manga,r,Pierrot
4,39486,Gintama: The Final,2021-01-08,2021-01-08,Two years have passed following the Tendoshuu'...,9.05,5.0,1550,140032,67119,movie,finished_airing,"Action,Comedy,Drama,Gag Humor,Historical,Parod...",1,winter,NaN,NaN,manga,pg_13,Bandai Namco Pictures


In [4]:
user_ratings_df.drop(columns=['start_date', 'finish_date'], inplace=True)

# Handling Missing Values

In [5]:
user_ratings_df.isna().sum()

user_id                 0
anime_id                0
score                   0
status                  1
num_episodes_watched    0
updated_at              0
dtype: int64

Although the above functions shows that there are no missing score values, a score of 0 indicates that the score was missing since it's not possible on MyAnimeList to rate an anime 0 unless you choose not to rate the anime.

## Remove Users who have only given scores of 0

In [6]:
users_to_remove = []
for user_id in user_ratings_df.user_id.unique():
  users_ratings = user_ratings_df.loc[user_ratings_df.user_id == user_id]
  num_non_zero_scores = len(users_ratings.loc[users_ratings.score > 0])

  if num_non_zero_scores == 0:
    users_to_remove.append(user_id)

print(users_to_remove)

[20, 35, 40, 57, 88, 116, 122, 124, 127, 131, 137, 139, 148, 185, 198, 202, 210, 217, 225, 226, 229, 232, 234, 235, 253, 271, 308, 309, 315, 321, 322, 364, 376, 384, 385, 387, 393, 419, 426, 443, 462, 476, 479, 487, 504, 506, 516, 532, 559, 562, 564, 569, 570, 590, 599, 612, 615, 617, 626, 644, 660, 664, 691, 723, 732, 733, 741, 745, 751, 761, 763, 766, 770, 771, 776, 785, 788, 794, 804, 815, 816, 844, 853, 860, 868, 879, 883, 886, 890, 897, 898, 906, 926, 961, 992, 999, 1023, 1029, 1033, 1054, 1059, 1065, 1077, 1090, 1099, 1105, 1108, 1114, 1127, 1129, 1143, 1145, 1147, 1156, 1157, 1170, 1180, 1223, 1227, 1228, 1230, 1237, 1258, 1268, 1271, 1272, 1273, 1310, 1314, 1351, 1358, 1370, 1386, 1387, 1388, 1392, 1406, 1409, 1420, 1427, 1446, 1455, 1458, 1469, 1477, 1490, 1491, 1511, 1522, 1529, 1541, 1542, 1549, 1557, 1570, 1576, 1587, 1594, 1596, 1604, 1607, 1616, 1641, 1650, 1652, 1653, 1655, 1662, 1667, 1691, 1693, 1715, 1720, 1732, 1746, 1752, 1755, 1764, 1766, 1795, 1806, 1808, 1840, 18

In [7]:
user_ratings_df = user_ratings_df.loc[~user_ratings_df.user_id.isin(users_to_remove)]

# Splitting User Ratings into Train and Test Datasets
Split based on the update_at column and save both files as CSVs.

In [8]:
import math
split_ratio = 0.7 # The ratio of anime per user to put in the training set

def train_test_split(user_ratings_df: pd.DataFrame, split_ratio: float):
  train_user_ratings = []
  test_user_ratings = []

  for user_id in user_ratings_df['user_id'].unique():
    # Get the users ratings and sort by the date the rating was last updated
    users_ratings = user_ratings_df.loc[user_ratings_df.user_id == user_id]
    users_ratings = users_ratings.sort_values(by=['updated_at'])

    # Determine what index to split the user's ratings by
    split_index = int(math.floor((len(users_ratings)-1) * split_ratio))

    # Append the split datasets
    train_user_ratings.append(users_ratings[:split_index])
    test_user_ratings.append(users_ratings[split_index+1:])

  train_df = pd.concat(train_user_ratings, ignore_index=True)
  test_df = pd.concat(test_user_ratings, ignore_index=True)
    
  return (train_df, test_df)

(train_df, test_df) = train_test_split(user_ratings_df, split_ratio)

train_df.to_csv('./data/train_user_ratings.csv', index=False)
test_df.to_csv('./data/test_user_ratings.csv', index=False)

In [9]:
train_df

,user_id,anime_id,score,status,num_episodes_watched,updated_at
0,0,48561,9,completed,1,2023-01-01 09:37:49+00:00
1,0,20583,9,completed,25,2023-01-01 09:39:58+00:00
2,0,38883,8,completed,13,2023-01-01 09:40:05+00:00
3,0,28891,10,completed,25,2023-01-01 09:40:24+00:00
4,0,32935,9,completed,10,2023-01-01 09:40:42+00:00
...,...,...,...,...,...,...
2912948,9071,35247,4,completed,7,2022-09-27 03:09:59+00:00
2912949,9071,31181,4,completed,12,2022-09-27 03:10:01+00:00
2912950,9071,29803,3,completed,13,2022-09-27 03:10:02+00:00
2912951,9071,34134,3,completed,12,2022-09-27 03:10:04+00:00


In [10]:
test_df

,user_id,anime_id,score,status,num_episodes_watched,updated_at
0,0,38826,9,completed,1,2023-04-15 14:38:56+00:00
1,0,40028,10,completed,16,2023-04-15 14:42:02+00:00
2,0,51535,9,watching,1,2023-04-15 14:42:16+00:00
3,0,48583,10,completed,12,2023-04-15 14:42:25+00:00
4,0,41467,10,completed,13,2023-04-15 14:42:51+00:00
...,...,...,...,...,...,...
1253543,9071,21,6,watching,1048,2023-08-19 14:33:51+00:00
1253544,9071,37171,5,completed,12,2023-09-07 12:21:07+00:00
1253545,9071,659,5,completed,1,2023-09-07 12:21:41+00:00
1253546,9071,35838,5,completed,12,2023-09-07 12:22:10+00:00


# Setup the Distance Functions

In [22]:
# Return a distance matrix using cosine distance
def cosine_distance_metric(user_profile, anime_vector_df):
    distance_mat = cosine_distances(np.array([user_profile["weighted_vector_avg"].tolist()]).reshape((1, -1)), anime_vector_df).reshape(-1)
    return distance_mat

# Return a distance matrix using euclidean distance
def euclidean_distance_metric(user_profile, anime_vector_df):
    distance_mat = euclidean_distances(np.array([user_profile["weighted_vector_avg"].tolist()]).reshape((1, -1)), anime_vector_df).reshape(-1)
    return distance_mat

def manhattan_distance_metric(user_profile, anime_vector_df):
    distance_mat = manhattan_distances(np.array([user_profile["weighted_vector_avg"].tolist()]).reshape((1, -1)), anime_vector_df).reshape(-1)
    return distance_mat

# Create the Recommender Class

In [23]:
from sklearn.metrics import mean_squared_error

class CBFRecommender:
  def __init__(self, anime_data: pd.DataFrame, user_ratings_data: pd.DataFrame):
    # Cleanup Anime Data
    self.anime_df = anime_data
    self.anime_df.fillna({"genres": ""}, inplace=True)
    self.anime_df.fillna({"synopsis": ""}, inplace=True)

    self.user_ratings_df = user_ratings_df

  # Uses TF-IDF to Vectorize the Anime DataFrame
  def vectorize_anime_data(self):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=50)
    self.anime_df['combined'] =  self.anime_df['genres'] + " " + self.anime_df['synopsis']

    anime_features_vector_matrix = vectorizer.fit_transform(self.anime_df['combined'])

    self.anime_vector_df = pd.DataFrame(data=anime_features_vector_matrix.toarray())
    self.anime_vector_df['anime_id'] = self.anime_df['id']
    self.anime_vector_df = self.anime_vector_df.set_index('anime_id')

    return self.anime_vector_df

  # Creates a profile of the specified user
  def create_user_profile(self, user_id: int):
    # Get only this user's ratings
    users_ratings_df = user_ratings_df[user_ratings_df.user_id == user_id]

    # Prefer to only use anime they rated higher than their avg. rating
    average_rating = np.average(users_ratings_df['score'])
    selected_user_ratings_df = users_ratings_df[users_ratings_df.score > average_rating]

    # If the user hasn't rated any anime higher than thier avg.,
    # then use the median instead
    if selected_user_ratings_df.empty:
      average_rating = np.median(users_ratings_df['score'])
      selected_user_ratings_df = users_ratings_df[users_ratings_df.score >= average_rating]

    # Get the Anime they rated highly
    user_anime_rated_df = self.anime_df[self.anime_df.id.isin(selected_user_ratings_df['anime_id'])]

    # Get the weighted average of the Vector values for the anime they rated
    user_vector_df = self.anime_vector_df.loc[user_anime_rated_df['id']]
    weighted_vector_avg = user_vector_df.mean()

    return {
        "weighted_vector_avg": weighted_vector_avg,
        "vector_df": user_vector_df
    }

  # Get a DataFrame of How Distant Anime Are to the User's Preferences
  def get_user_anime_distance(self, user_id, distance_metric='cosine'):
    # Find user profile in user profiles
    user_profile = self.create_user_profile(user_id)

    # Calculate the distance matrix of the user's weighted vector instance compared to all instances in the anime vector df
    distance_mat = None
    if distance_metric == 'cosine':
        distance_mat = cosine_distance_metric(user_profile, self.anime_vector_df)
    elif distance_metric == 'euclidean':
        distance_mat = euclidean_distance_metric(user_profile, self.anime_vector_df)
    elif distance_metric == 'manhattan':
        distance_mat = manhattan_distance_metric(user_profile, self.anime_vector_df)
    else:
        print(f'Error: The distance metric is invalid. Unable to create distance matric for user with id {user_id}')

    if distance_mat is None:
        return None
    
    # Convert the matrix into a dataframe
    distance_df = pd.DataFrame(data=distance_mat.tolist())
    distance_df = distance_df.rename(columns={0: "distance"})
    distance_df['id'] = self.anime_df['id']

    # Remove all anime that the user has already watched
    distance_df = distance_df.loc[~distance_df.id.isin(user_profile["vector_df"].index)]
    
    # The closer to 0 the distance is the more similar the anime is to the weighted vector
    distance_df = distance_df.sort_values(by='distance', ascending=True)

    return distance_df

  # Recommend Anime to the User
  def recommend_user(self, user_id: int, num_recommendations: int, distance_metric='cosine', add_anime_info: bool = True):
    distance_df = self.get_user_anime_distance(user_id, distance_metric=distance_metric)

    # Get the top recommendations and merge the anime data with the dataframe
    top_similar_anime_df = distance_df.iloc[0:num_recommendations]
    
    if len(top_similar_anime_df) != num_recommendations:
        print(f"Expected {num_recommendations} recommendations, but got {len(top_similar_anime_df)}")

    # Only merge if we want to the anime information too.
    if add_anime_info:
      top_similar_anime_df = top_similar_anime_df.merge(self.anime_df, on='id', how='inner')

    return top_similar_anime_df

  # Test How Effective the Model
  def test_recommendations(self, distance_metric='cosine'):
    unique_user_ids = self.user_ratings_df['user_id'].unique()
    
    actual_anime = []
    predicted_anime = []
    
    for user_id in unique_user_ids:
      # Get this user's recommendations
      actual_anime_vectors = self.anime_vector_df.loc[self.anime_vector_df.index.isin(test_df.loc[test_df.user_id == user_id]['anime_id'])]
      num_recommendations = len(actual_anime_vectors)
    
      if num_recommendations > 0:
        users_top_similar_anime_df = self.recommend_user(user_id, num_recommendations, distance_metric=distance_metric, add_anime_info = False)
        
        for anime in actual_anime_vectors.values:
          actual_anime.append(anime.tolist())
        
        predicted_anime_vectors = self.anime_vector_df.loc[self.anime_vector_df.index.isin(users_top_similar_anime_df['id'])]
        
        if len(actual_anime_vectors) != len(predicted_anime_vectors):
            print(f"Expected {len(actual_anime_vectors)} predictions, but got {len(predicted_anime_vectors)}")
        
        for anime in predicted_anime_vectors.values:
            predicted_anime.append(anime.tolist())
        
    return {
        "rmse": mean_squared_error(actual_anime, predicted_anime, squared=False),
    }

In [24]:
anime_cbf_recommeder = CBFRecommender(anime_df, train_df)
anime_cbf_recommeder.vectorize_anime_data()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
anime_id,,,,,,,,,,,,,,,,,,,,,
5114,0.243555,0.264325,0.0,0.0,0.000000,0.0,0.293202,0.000000,0.0,0.235457,...,0.0,0.00000,0.000000,0.0,0.0,0.296467,0.276524,0.0,0.365017,0.0
9253,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.208083,0.000000,0.0,0.000000,...,0.0,0.00000,0.708988,0.0,0.0,0.000000,0.196247,0.0,0.000000,0.0
28977,0.191855,0.000000,0.0,0.0,0.166754,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.64006,0.000000,0.0,0.0,0.233535,0.217825,0.0,0.000000,0.0
41467,0.221499,0.240388,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.214134,...,0.0,0.00000,0.000000,0.0,0.0,0.539239,0.251482,0.0,0.000000,0.0
39486,0.201203,0.000000,0.0,0.0,0.174879,0.0,0.242216,0.331013,0.0,0.000000,...,0.0,0.00000,0.275096,0.0,0.0,0.244914,0.228439,0.0,0.301544,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56512,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
56779,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
53029,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


## Test Recommending Anime for One User

In [31]:
test_user_id = 1
test_user_recommendations = anime_cbf_recommeder.recommend_user(test_user_id, 10)
test_user_recommendations.head()

,distance,id,title,start_date,end_date,synopsis,score,rank,popularity,num_list_users,...,status,genres,num_episodes,start_season,broadcast_day,broadcast_time,source,rating,studios,combined
0,0.212616,2787,Shakugan no Shana II (Second),2007-10-05,2008-03-28,Denizens from the Crimson Realm continue to in...,7.55,1602.0,751,305281,...,finished_airing,"Action,Drama,Fantasy,Mahou Shoujo,Romance,School",24,fall,friday,01:25,light_novel,pg_13,J.C.Staff,"Action,Drama,Fantasy,Mahou Shoujo,Romance,Scho..."
1,0.239674,39533,Given,2019-07-12,2019-09-20,"Tightly clutching his Gibson guitar, Mafuyu Sa...",8.32,231.0,381,549672,...,finished_airing,"Boys Love,Drama,Music,School",11,summer,friday,00:55,manga,pg_13,Lerche,"Boys Love,Drama,Music,School Tightly clutching..."
2,0.239795,35466,ReLIFE: Kanketsu-hen,2018-03-21,2018-03-21,After reliving the life of a high school stude...,8.20,355.0,585,380855,...,finished_airing,"Drama,Romance,School",4,winter,NaN,NaN,web_manga,pg_13,TMS Entertainment,"Drama,Romance,School After reliving the life o..."
3,0.240957,11843,Danshi Koukousei no Nichijou,2012-01-10,2012-03-27,Roaming the halls of the all-boys Sanada North...,8.23,309.0,214,805340,...,finished_airing,"Comedy,Gag Humor,School",12,winter,tuesday,02:00,web_manga,pg_13,Sunrise,"Comedy,Gag Humor,School Roaming the halls of t..."
4,0.246221,38027,Free! Dive to the Future - Soushun no Build-up!,2018-06-23,2018-06-23,"As they adjust to life in Tokyo, Haruka Nanase...",7.55,1622.0,4458,22471,...,finished_airing,"Comedy,School",1,spring,NaN,NaN,original,pg_13,"Kyoto Animation,Animation Do","Comedy,School As they adjust to life in Tokyo,..."


## Evaluating the CBF Recommender on Different Distance Metrics

In [21]:
test_recommendations_cosine = anime_cbf_recommeder.test_recommendations(distance_metric='cosine')
test_recommendations_cosine

{'rmse': 0.15124028899865474}

In [20]:
test_recommendations_euclidean = anime_cbf_recommeder.test_recommendations(distance_metric='euclidean')
test_recommendations_euclidean

{'rmse': 0.13675863362062837}

In [32]:
test_recommendations_manhattan = anime_cbf_recommeder.test_recommendations(distance_metric='manhattan')
test_recommendations_manhattan

{'rmse': 0.1397449250720996}